## ML

In [1]:
import pandas as pd

Import & Modify Data

In [2]:
dataset = pd.read_csv(r'/home/utilisateur/Documents/DEV-IA/projets/BOX/eda/ml_data.csv')
dataset.sample(1)

,Unnamed: 0,title,release_year,original_title,duration_m,synopsis,all_director_oscars,all_actor_oscars,actor_celebs,actor_celebs_by_year,...,entries_mean_composer,entries_sum_composer,jpbox_copies,entries,genre,production,country,director,title_length,synopsis_length
41759,41759,Un espion ordinaire,2021,NaN,112.0,1960. Modeste représentant de commerce anglais...,NaN,NaN,NaN,NaN,...,0.0,0.0,323.0,103555,Espionnage,FilmNation Entertainment,Grande-Bretagne,Dominic Cooke,19,493


Import Librairies

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from catboost import CatBoostRegressor, Pool

Separate Features & Target

In [4]:
dataset = dataset.drop(['Unnamed: 0'], axis=1)

In [5]:
X = dataset.copy()
y = X.pop('entries')

In [6]:
columns = ['feature_'+str(i) for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=columns)
df.shape

(47543, 22)

Separate Data into Train/Test sets

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [8]:
numerical_cols = ['release_year', 'duration_m', 'all_director_oscars', 'all_actor_oscars', 'actor_celebs', 'actor_celebs_by_year', 'entries_mean_actor', 'entries_sum_actor', 'entries_mean_director', 'entries_sum_director', 'entries_mean_composer', 'entries_sum_composer', 'jpbox_copies']
categorical_cols = ['title', 'original_title', 'genre', 'synopsis', 'production', 'country', 'director']

In [9]:
X_train[categorical_cols] = X_train[categorical_cols].fillna("NaN")
X_test[categorical_cols] = X_test[categorical_cols].fillna("NaN")

In [10]:
feature_names = list(X_train.columns)

In [11]:
train_pool = Pool(data=X_train, label=y_train, feature_names=feature_names, cat_features=categorical_cols)
test_pool = Pool(data=X_test, label=y_test, feature_names=feature_names, cat_features=categorical_cols)

Features Preprocessing

In [12]:
X.columns

Index(['title', 'release_year', 'original_title', 'duration_m', 'synopsis',
       'all_director_oscars', 'all_actor_oscars', 'actor_celebs',
       'actor_celebs_by_year', 'entries_mean_actor', 'entries_sum_actor',
       'entries_mean_director', 'entries_sum_director',
       'entries_mean_composer', 'entries_sum_composer', 'jpbox_copies',
       'genre', 'production', 'country', 'director', 'title_length',
       'synopsis_length'],
      dtype='object')

In [13]:
preprocessing = ColumnTransformer(
    transformers = [
        ("num", StandardScaler(), numerical_cols)
    ],
    remainder="passthrough"
)

preprocessing.set_output(transform="default")

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', StandardScaler(),
                                 ['release_year', 'duration_m',
                                  'all_director_oscars', 'all_actor_oscars',
                                  'actor_celebs', 'actor_celebs_by_year',
                                  'entries_mean_actor', 'entries_sum_actor',
                                  'entries_mean_director',
                                  'entries_sum_director',
                                  'entries_mean_composer',
                                  'entries_sum_composer', 'jpbox_copies'])])

In [14]:
X_train_cb = preprocessing.fit_transform(X_train)

Choose Best Model

In [18]:
from catboost import CatBoostClassifier
catboost = CatBoostRegressor(cat_features=categorical_cols, eval_metric='RMSE')

In [19]:
from sklearn.pipeline import make_pipeline
model = make_pipeline(preprocessing, catboost)

In [20]:
from sklearn.metrics import root_mean_squared_error
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
raw_score = root_mean_squared_error(y_test, y_pred, average="macro")
raw_score

CatBoostError: features parameter contains string value 'title' but feature names for a dataset are not specified